# log plot attempts

In [ ]:
import sys
import numpy, matplotlib, pandas
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
ts, bnn, ts_sorted, Bundeslaender_sorted, dates, datacolumns = dataMangling.dataMangled()

In [ ]:
dataPlotting.test_plot_Kreis(ts=ts, bnn=bnn, dates=dates, datacolumns=datacolumns)

# this following new code could be dropped in place ...

where the old "plot_timeseries()" is.

Because the defaults (`logPlot=False, unsmoothedAlso=True`) will make it behave exactly like the old code.

But switching to (`logPlot=True, unsmoothedAlso=False`) then shows the (7 days, 14 days) smoothed averages on a loglin plot.

In [ ]:
from matplotlib import pyplot as plt
import os, datetime
import pandas
import matplotlib 

def plot_timeseries(datacolumns, dates, daily, cumulative, title, filename, 
                    ifShow=True, ifCleanup=True, population=None, limitIncidencePerWeekPerMillion=500,
                    logPlot=False, unsmoothedAlso=True):

    fig, ax = plt.subplots(figsize=(10, 6)) #, constrained_layout=True)
    # plt.tight_layout()

    # x axis
    ax.xaxis_date()
    ax.xaxis.set_minor_locator(matplotlib.dates.DayLocator())
    fig.autofmt_xdate(rotation=60)
    
    if logPlot:
        plotter=ax.semilogy
        plt.ylim(0.1, max(daily[1:])*1.5)
        markerY = 0.13
    else:
        plotter=ax.plot
        plt.ylim(0, max(daily[1:])*1.5)
        markerY = max(daily[1:])/19

    # plot data
    if unsmoothedAlso:
        lns1 = plotter(dates, daily, label="daily cases (weekend-flawed), 2 weeks: red", color='lightgray')
        lns1_2 = plotter(dates[-14:], daily[-14:], label="daily cases, last 14 days dark gray", color='red')
        # print (len(dates[-14:]))
    else:
        lns1, lns1_2 = [],[]
    
    plt.ylabel("daily cases", color="purple")
    
    # plot averages
    window=7
    rolling_mean = pandas.DataFrame(daily).rolling(window=window, center=True).mean()
    lns2 = plotter(dates, rolling_mean, label='daily: centered moving average %s days' % window, color='purple')
    window=14
    rolling_mean = pandas.DataFrame(daily).rolling(window=window, center=True).mean()
    lns3 = plotter(dates, rolling_mean, label='daily: centered moving average %s days' % window, color='orange', linewidth=4)
    # window=21
    # rolling_mean = pandas.DataFrame(daily).rolling(window=window, center=True).mean()
    # ax.plot(dates, rolling_mean, label='SMA %s days' % window, color='pink', linewidth=1)

    # plot center bar
    center, signal = dataMangling.temporal_center(daily)
    # print (center)
    center_date=datacolumns.values[int(round(center))]
    # lns4 = ax.bar(dates, signal, label="'expectation day': "+center_date, color='green')
    
    # lns4_2 = plt.plot(dates[int(round(center))], max(signal), marker="v", color='green', markersize=15)
    # lns4_2 = plt.plot(dates[int(round(center))], 0, marker="v", color='green', markersize=15)
    # lns4_2 = plt.plot(dates[int(round(center))], [max(daily[1:])/20], marker="^", color='green', markersize=30)
    lns4_2 = plt.plot(dates[int(round(center))], [markerY], marker="v", color='green', markersize=17)


    # plot 2nd axis and cumulative data
    ax2 = plt.twinx()
    plt.ylim(0, max(cumulative)*1.1)
    
    plt.ylabel("cumulative total cases", color="#1E90FF")

    lns5 = ax2.plot(dates, cumulative, label="total cases reported at RiskLayer", color = '#1E90FF')
    
    lns6 = []
    if population:
        limit = limitIncidencePerWeekPerMillion/7*population/1000000
        # print ("limit:", limit)
        lns6 = ax.plot([dates[1]]+[dates[-1]],[limit,limit], label="daily %.2f =limit 500/week/1mio pop." % limit, color = '#ef7c7c', linestyle=  (0, (5, 10)))

    lines = lns5 + lns1 + lns2 + lns3 + lns6
    labs = [l.get_label() for l in lines]

    text = "source data @RiskLayer up to " + ("%s"%max(dates))[:10]
    text += "\nplot @DrAndreasKruger " + ("%s" % datetime.datetime.now())[:16]
    text += "\ndaily: (GREEN) 'expectation day' = "+center_date

    plt.legend(lines, labs, loc='upper left', facecolor="#fafafa", framealpha=0.8, 
               title=text, prop={'size': 8}, title_fontsize = 8)

    plt.title(title)
    
    if filename:
        fig.savefig(os.path.join(dataFiles.PICS_PATH, filename),  bbox_inches='tight')
    
    if ifShow:
        plt.show()
        
    if ifCleanup:
        plt.clf()
        plt.close("all")
    
    return plt, fig, ax, ax2

def test_plot_Kreis(ts, bnn, dates, datacolumns, logPlot=False, unsmoothedAlso=True):
    ## Kreis
    AGS = "0"
    #AGS = "1001"
    AGS = "5370"
    # AGS = "9377"
    daily, cumulative, title, filename, pop = dataMangling.get_Kreis(ts, bnn, AGS)
    plot_timeseries(datacolumns, dates, daily, cumulative, title, filename=filename, population=pop,
                    logPlot=logPlot, unsmoothedAlso=unsmoothedAlso)

test_plot_Kreis(ts=ts, bnn=bnn, dates=dates, datacolumns=datacolumns, logPlot=False, unsmoothedAlso=True)

In [ ]:
test_plot_Kreis(ts=ts, bnn=bnn, dates=dates, datacolumns=datacolumns, unsmoothedAlso=False, logPlot=True)

# now the same for many Kreise

In [ ]:
AGSs=list(ts["AGS"])[0:5]
AGSs

In [ ]:
def plotSeveralAGSs(AGSs, ts, bnn, dates, datacolumns, logPlot=False, unsmoothedAlso=True):
    for AGS in AGSs:
        daily, cumulative, title, filename, pop = dataMangling.get_Kreis(ts, bnn, AGS)
        print(AGS, title)
        plot_timeseries(datacolumns, dates, daily, cumulative, title, filename=filename, population=pop,
                        logPlot=logPlot, unsmoothedAlso=unsmoothedAlso)
        
plotSeveralAGSs(AGSs, ts=ts, bnn=bnn, dates=dates, datacolumns=datacolumns,
                logPlot=True, unsmoothedAlso=False)
#               logPlot=False, unsmoothedAlso=True)